<span style='font-family:"Times New Roman"'>

## Bibliotecas

In [1]:
#Básico
import sqlite3 
import pandas as pd
import numpy as np 
from scipy.stats import mstats
from dotenv import dotenv_values
from datetime import datetime


#Pré processamento e otimização
from sklearn.model_selection import train_test_split
from ModelUtitls import Preprocessor
from ModelUtitls import SelectModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,KFold


# Modelos
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.kernel_ridge import KernelRidge

import pickle

# Disabilitarv avisos
import warnings
warnings.filterwarnings('ignore')

<span style='font-family:"Times New Roman"'>

## Configurações

In [2]:
# Ler arquivo de configurações
config = dotenv_values("../.env")

# Versão
version = config["VERSION"]

# Aleatoriedade "padrão"
seed = 42
np.random.seed(seed=seed)

# Leitura de dados
pd.set_option('display.max_columns', 20)


<span style='font-family:"Times New Roman"'>

## Conexões

In [3]:
#Conectar com o banco de dados
connection = sqlite3.connect(config["DB_CONFIG"])

In [4]:
#Ler
df_numeric = pd.read_sql_query(f"SELECT * FROM numerical_features_{version}", connection)
df_categor = pd.read_sql_query(f"SELECT * FROM categorical_features_{version}", connection)

df_numeric.drop(columns=["index"],inplace=True)
df_categor.drop(columns=["index"],inplace=True)

df = df_categor.join(df_numeric)
df.head()

,Neighborhood,ExterQual,BsmtQual,HeatingQC,CentralAir,KitchenQual,GarageFinish,OverallQual,YearBuilt,MasVnrArea,...,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,GarageYrBlt,GarageCars,GarageArea,SalePrice
0,CollgCr,Gd,Gd,Ex,Y,Gd,RFn,7,2003,196.0,...,856,856,854,1710,2,8,2003.0,2,548,208500
1,Veenker,TA,Gd,Ex,Y,TA,RFn,6,1976,0.0,...,1262,1262,0,1262,2,6,1976.0,2,460,181500
2,CollgCr,Gd,Gd,Ex,Y,Gd,RFn,7,2001,162.0,...,920,920,866,1786,2,6,2001.0,2,608,223500
3,Crawfor,TA,TA,Gd,Y,Gd,Unf,7,1915,0.0,...,756,961,756,1717,1,7,1998.0,3,642,140000
4,NoRidge,Gd,Gd,Ex,Y,Gd,RFn,8,2000,350.0,...,1145,1145,1053,2198,2,9,2000.0,3,836,250000


In [5]:
# Variável Objetivo
y = df["SalePrice"]
X = df.drop(columns=["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)


In [6]:
preprocessor = Preprocessor(X).transformer()
models = {"Gradient Boosting": GradientBoostingRegressor(),
          "XG Boost": XGBRegressor(seed=seed),
          "Elastic Net": ElasticNet(),
          "Linear Regression": LinearRegression(),
          "Tree Regressor": DecisionTreeRegressor(random_state=seed),
          "LGBM Regressor": LGBMRegressor(),
          "Kernel Ridge": KernelRidge()}


In [7]:
avaliation = SelectModel(models, X_train, y_train, seed, preprocessor).guess_best()

avaliation_table = pd.DataFrame(avaliation)
avaliation_table.sort_values(by=['r_square_mean'], ascending=False, inplace=True)
avaliation_table.reset_index(drop=True, inplace=True)

avaliation_table


,model,r_square_mean,r_square_stdv,models_algor
0,Gradient Boosting,0.850603,0.071686,GradientBoostingRegressor()
1,LGBM Regressor,0.841636,0.052826,LGBMRegressor()
2,XG Boost,0.825126,0.088346,"XGBRegressor(base_score=None, booster=None, ca..."
3,Kernel Ridge,0.807142,0.101357,KernelRidge()
4,Linear Regression,0.805479,0.099232,LinearRegression()
5,Elastic Net,0.768100,0.136360,ElasticNet()
6,Tree Regressor,0.721169,0.096645,DecisionTreeRegressor(random_state=42)


In [8]:
best_alg = avaliation_table.models_algor.head(1)
best_alg = Pipeline(steps=[('preprocessor', preprocessor), ('model', best_alg[0])])
best_alg.fit(X_train, y_train)

pickle.dump(best_alg, open(f'{config["DEPLOY_VOL"]}_{version}.pkl','wb'))